In [1]:
import os, sys
import glob
import json
import re
from tqdm import tqdm 
from itertools import product

import numpy as np
import pandas as pd

sys.path.append('/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/utils/')

from config import *
import analysis_utils as utils

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /dartfs/rc/lab/F/FinnLab/tommy/models/token
Login successful


2025-03-29 13:33:00.372888: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-29 13:33:00.372952: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-29 13:33:00.372990: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-29 13:33:00.383480: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-29 13:33:09.197277: W tensorflow/compiler/

In [2]:
task_list = ['black', 'wheretheressmoke', 'howtodraw']
preproc_dir = os.path.join(BASE_DIR, 'stimuli/preprocessed')

In [20]:
story_stats = {}

for task in task_list:
    selected_fn = glob.glob(os.path.join(preproc_dir, task, '*selected.csv'))[0]
    df_selected = pd.read_csv(selected_fn)

    n_sampled = df_selected['NWP_Candidate'].sum()
    total_words = len(df_selected['NWP_Candidate'])
    percent_words = 100 * (n_sampled / total_words)

    story_stats[task] = {
        'n_sampled': n_sampled,
        'total_words': total_words,
        'percent_words': percent_words,
        'percent_words_subj': percent_words / 3 if task != 'black' else percent_words / 4
    }

In [25]:
total_sampled_words = [story_stats[task]['n_sampled'] for task in task_list]

all_task_percent_words = [story_stats[task]['percent_words'] for task in task_list]
all_task_percent_subj = [story_stats[task]['percent_words_subj'] for task in task_list]

print (f"Total words sampled: {sum(total_sampled_words)} ")
print (f"Average percent of all words: {np.mean(all_task_percent_words):.2f} ")
print (f"Average percent of subj words: {np.mean(all_task_percent_subj):.2f} ")

Total words sampled: 672 
Average percent of all words: 12.64 
Average percent of subj words: 3.86 


In [ ]:
[story_stats[for k, v in story_stats.items()]

{'black': {'n_sampled': 194,
  'total_words': 1541,
  'percent_words': 0.12589227774172615},
 'wheretheressmoke': {'n_sampled': 194,
  'total_words': 1541,
  'percent_words': 0.12589227774172615},
 'howtodraw': {'n_sampled': 194,
  'total_words': 1541,
  'percent_words': 0.12589227774172615}}

In [10]:
df_selected = pd.read_csv(selected_fn)

In [15]:
df_selected['NWP_Candidate'].sum() / len(df_selected)

0.12589227774172615

In [126]:
def get_subject_demographics(results_dir, task, modality):

    if task == 'black':
        results_dir = os.path.join(results_dir, 'pilot-multimodal-01')
    else:
        results_dir = os.path.join(results_dir, 'final-multimodal-01')

    # Get subject directories
    sub_dirs = sorted(glob.glob(os.path.join(results_dir, task, modality, f'sub*')))

    print(f'Total of {len(sub_dirs)} subjects')

    all_demographics = []

    for sub_dir in tqdm(sub_dirs):
        sub = os.path.basename(sub_dir)
        current_id, demographics, _, _ = utils.load_participant_results(sub_dir, sub)

        age_filter = demographics['experiment_phase'] == 'demographics-age'

        try: 

            current_age = demographics.loc[age_filter, 'response'].values.astype(float)

            if current_age < 65:
                demographics.loc[age_filter, 'response'] = current_age
            else:
                demographics.loc[age_filter, 'response'] = np.nan
        except:
            demographics.loc[age_filter, 'response'] = np.nan

        df_demographics = demographics.T.reset_index(drop=True)
        df_demographics.columns = df_demographics.iloc[0]
       
       # Remove the first row
        df_demographics = df_demographics.iloc[1:]
        df_demographics['prolific_id'] = current_id

        all_demographics.append(df_demographics)

    df_demographics = pd.concat(all_demographics).reset_index(drop=True)
    return df_demographics

In [134]:
all_demographics['demographics-age'] = all_demographics['demographics-age'].astype(float)

In [ ]:
task_list = ['black', 'wheretheressmoke', 'howtodraw']
modality_list = ['video', 'audio', 'text']

results_dir = os.path.join(BASE_DIR, 'experiments',  'next-word-prediction', 'cleaned-results')

all_demographics = []

for task, modality in product(task_list, modality_list):
    demographics = get_subject_demographics(results_dir=results_dir, task=task, modality=modality)
    all_demographics.append(demographics)

all_demographics = pd.concat(all_demographics).reset_index(drop=True)
all_demographics['demographics-age'] = all_demographics['demographics-age'].astype(float)

Total of 200 subjects


100%|██████████| 200/200 [00:04<00:00, 40.71it/s]


Total of 200 subjects


100%|██████████| 200/200 [00:05<00:00, 39.05it/s]


Total of 200 subjects


...

100%|██████████| 150/150 [00:04<00:00, 36.59it/s]


Total of 150 subjects


100%|██████████| 150/150 [00:03<00:00, 39.95it/s]


Total of 150 subjects


100%|██████████| 150/150 [00:04<00:00, 34.59it/s]


### Gender demographics

In [137]:
all_demographics['demographics-gender'].value_counts()

Female                   743
Male                     727
Do not wish to report     14
Non-Binary                10
Other                      6
Name: demographics-gender, dtype: int64

### Age demographics

In [135]:
mean_age = all_demographics['demographics-age'].mean()
std_age = all_demographics['demographics-age'].std()

print (f"Mean age: {mean_age}")
print (f"STD age: {std_age}")

Mean age: 37.78812415654521
STD age: 11.791443096895717
